# Test all faults and cases

This Notebook shows all Tests that are implemented in the sce_test/test_all_faults_and_cases.py file. Therefor it must be run in an environment where the pandapower test folder can be found by the jupyter server's python environment.

In [1]:
import os
from itertools import product
import numpy as np

from pandapower import pp_dir
from pandapower.file_io import from_json
from pandapower.test.shortcircuit.SCE_Tests.test_all_faults_and_cases import load_pf_results, get_columns_to_check, modify_impedance_values_with_fault_value, check_pattern
from pandapower.shortcircuit import calc_sc

testfiles_path = os.path.join(pp_dir, 'test', 'shortcircuit', 'sce_tests')

From the defined parameters for faults, cases, network names and vector_groups as well as the values for r_faults and x_faults all combinations are generated to run as a test.

In [2]:
# Define common parameters
faults = ["LLL", "LL", "LG", "LLG"]
cases = ["max", "min"]
values = [(0.0, 0.0), (5.0, 5.0)]
net_names = ["test_case_1_four_bus_radial_grid", "test_case_2_five_bus_radial_grid", "test_case_3_five_bus_meshed_grid"]
vector_groups = ['Dyn', 'Yyn', 'YNyn']

# Create parameter list
parametrize_values = product(faults, cases, values, net_names, vector_groups)

Then each of the 144 combinations is run as a test using the test_all_faults_and_cases_with_fault_impedance function.

we will use the first of these combinations to show how the tests are performed.

In [3]:
fault, case, (r_fault_ohm, x_fault_ohm), net_name, vector_group = next(parametrize_values)

The test will load the expected results from the corresponding excel file.
This will create a dictionary where each sheet from the excel is included under its name as a pandas DataFrame.
Below is the LLL_max sheet shown as an example.

In [4]:
if net_name != "test_case_1_four_bus_radial_grid":
    net_name += "_" + vector_group.lower()

net = from_json(os.path.join(testfiles_path, "test_grids", net_name + ".json"))

excel_file = os.path.join(testfiles_path, "sc_result_comparison", net_name + "_pf_sc_results_all_cases.xlsx")
dataframes = load_pf_results(excel_file)

dataframes['LLL_max']

,name,ikss_ka,skss_mw,rk_ohm,xk_ohm
0,Bus_0,2.886752,100.000006,0.437816,4.378163
1,Bus_1,2.510392,86.962535,0.957816,4.968163
2,Bus_2,2.208504,76.504825,1.477816,5.558163
3,Bus_3,2.208504,76.504825,1.477816,5.558163


Defined next is the precision that is expected for the results.

In [5]:
rtol = {"ikss_ka": 0, "skss_mw": 0, "rk_ohm": 0, "xk_ohm": 0}
atol = {"ikss_ka": 1e-6, "skss_mw": 1e-4, "rk_ohm": 1e-6, "xk_ohm": 1e-6}

A list of columns that should be verified is created as well as the corresponding dataframe selected.

In [6]:
columns_to_check = get_columns_to_check(fault)
selected_sheet = f"{fault}_{case}"
if r_fault_ohm != 0.0 and x_fault_ohm != 0.0:
    selected_sheet += "_fault"

selected_pf_results = dataframes[selected_sheet]

print(f"{columns_to_check=}\n{selected_sheet=}")

selected_pf_results


columns_to_check=['ikss_ka', 'skss_mw', 'rk_ohm', 'xk_ohm']
selected_sheet='LLL_max'


,name,ikss_ka,skss_mw,rk_ohm,xk_ohm
0,Bus_0,2.886752,100.000006,0.437816,4.378163
1,Bus_1,2.510392,86.962535,0.957816,4.968163
2,Bus_2,2.208504,76.504825,1.477816,5.558163
3,Bus_3,2.208504,76.504825,1.477816,5.558163


The impedance values are modified using the fault values. For the case selected here the `selected_pf_results` and `modified_pf_results` do not differ as `r_fault_ohm` and `x_fault_ohm` are 0. The function `modify_impedance_values_with_fault_value` will add the `r_fault_ohm` or `x_fault_ohm` value to the corresponding columns.

The dataframe is then sorted by name to ensure same order as results from short circuit calculation.

In [7]:
modified_pf_results = modify_impedance_values_with_fault_value(selected_pf_results, r_fault_ohm, x_fault_ohm)
modified_pf_results.sort_values(by='name', inplace=True)

modified_pf_results

,name,ikss_ka,skss_mw,rk_ohm,xk_ohm
0,Bus_0,2.886752,100.000006,0.437816,4.378163
1,Bus_1,2.510392,86.962535,0.957816,4.968163
2,Bus_2,2.208504,76.504825,1.477816,5.558163
3,Bus_3,2.208504,76.504825,1.477816,5.558163


Next the short circuit calculation is performed using pandapowers `calc_sc` method with the required parameters.

For the comparison the names of the buses are added to the results tables and values sorted by the bus names.

In [8]:
calc_sc(net, fault=fault, case=case, branch_results=True, return_all_currents=True, ip=False,
        r_fault_ohm=r_fault_ohm, x_fault_ohm=x_fault_ohm)

net.res_bus_sc.insert(0, "name", net.bus.name)
net.res_bus_sc.sort_values(by='name', inplace=True)

net.res_bus_sc

Branch results are in beta mode and might not always be reliable, especially for transformers


,name,ikss_ka,skss_mw,rk_ohm,xk_ohm
3,Bus_0,2.886751,100.000000,0.437816,4.378164
0,Bus_1,2.510392,86.962530,0.957816,4.968164
1,Bus_2,2.208504,76.504821,1.477816,5.558164
2,Bus_3,2.208504,76.504821,1.477816,5.558164


Now the tables are compared using `np.isclose` and the specified relative and absoulte tolerances.

In [9]:
for column in columns_to_check:
    column_ar = check_pattern(column)
    assert np.isclose(
        net.res_bus_sc.loc[:, column],
        modified_pf_results.loc[:, column],
        rtol=rtol[column_ar], atol=atol[column_ar]
    ).all()